In [ ]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# nltk.download()
from nltk.stem import WordNetLemmatizer
# from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

In [ ]:
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# New Section

In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

In [ ]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
training = []
print(classes)
outputEmpty = [0] * len(classes)

# outputEmpty = [0] * len(classes)

['admission', 'canteen', 'college intake', 'committee', 'computerhod', 'course', 'creator', 'document', 'event', 'extchod', 'facilities', 'fees', 'floors', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'ithod', 'library', 'location', 'menu', 'name', 'number', 'placement', 'principal', 'ragging', 'random', 'salutaion', 'scholarship', 'sem', 'sports', 'swear', 'syllabus', 'task', 'uniform', 'vacation']


In [ ]:
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

In [ ]:
random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

In [ ]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
hist = model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)

Epoch 1/200
81/81 [==============================] - 1s 3ms/step - loss: 3.5937 - accuracy: 0.0617
Epoch 2/200
81/81 [==============================] - 0s 2ms/step - loss: 3.4212 - accuracy: 0.1284
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.1916 - accuracy: 0.1778
Epoch 4/200
81/81 [==============================] - 0s 2ms/step - loss: 2.9075 - accuracy: 0.2593
Epoch 5/200
81/81 [==============================] - 0s 2ms/step - loss: 2.6757 - accuracy: 0.3086
Epoch 6/200
81/81 [==============================] - 0s 2ms/step - loss: 2.3545 - accuracy: 0.3951
Epoch 7/200
81/81 [==============================] - 0s 2ms/step - loss: 2.1859 - accuracy: 0.4346
Epoch 8/200
81/81 [==============================] - 0s 2ms/step - loss: 2.0192 - accuracy: 0.4543
Epoch 9/200
81/81 [==============================] - 0s 2ms/step - loss: 1.8424 - accuracy: 0.5037
Epoch 10/200
81/81 [==============================] - 0s 3ms/step - loss: 1.7016 - accuracy: 0.5506
Epoch 11/

In [ ]:
model.save('chatbot_model.h5', hist)
print('Done')

Done


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open("words.pkl", 'rb'))
classes = pickle.load(open("classes.pkl", 'rb'))
model = load_model('chatbot_model.h5')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i]=1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r> ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

1/1 [==============================] - 0s 178ms/step
Hi there, how can I help?
1/1 [==============================] - 0s 21ms/step
College students
1/1 [==============================] - 0s 21ms/step
Good to see you again!
1/1 [==============================] - 0s 21ms/step
College is open 8am-5pm Monday-Saturday!
1/1 [==============================] - 0s 22ms/step
Hello!
1/1 [==============================] - 0s 22ms/step
College students
1/1 [==============================] - 0s 34ms/step
Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.
